In [28]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

class PIDController:
    def __init__(self, Kp, Ki, Kd, min_output, max_output):
        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd
        self.min_output = min_output
        self.max_output = max_output
        self.cumulative_error = 0
        self.previous_error = 0

    def update(self, error, dt):
    
        # Proportional term
        P = self.Kp * error

        # Integral term
        self.cumulative_error += error * dt
        I = self.Ki * self.cumulative_error

        # Derivative term
        error_rate = (error - self.previous_error) / dt
        D = self.Kd * error_rate

        # Compute the control signal
        control = P + I + D

        # Clamp the output
        control = max(self.min_output, min(control, self.max_output))

        # Update previous error
        self.previous_error = error

        return control

class VehicleDynamics:
    def __init__(self, mass, min_accel, max_accel):
        self.mass = mass
        self.min_accel = min_accel
        self.max_accel = max_accel
        # Initialize other parameters specific to the vehicle dynamics

    # def update_state(self, x, vx, y, vy, force_x, force_y, dt):
    #     # Implement the dynamics here
    #     # For example, a simple point mass dynamics could be:
    #     ax = force_x / self.mass
    #     ay = force_y / self.mass

    #     # Update position and velocity (using Verlet or other integration method)
    #     # ...

    #     return new_x, new_vx, new_y, new_vy
    
    def update_state_verlet(self, x, vx, y, vy, ax, ay, dt):
        # Update position using Velocity Verlet
        x_new = x + vx * dt + 0.5 * ax * dt**2
        y_new = y + vy * dt + 0.5 * ay * dt**2

        # Predict the next velocity (half step)
        vx_half = vx + 0.5 * ax * dt
        vy_half = vy + 0.5 * ay * dt

        # Update acceleration here based on new position if needed
        # For constant acceleration, this step can be skipped

        # Complete the velocity update (second half step)
        vx_new = vx_half + 0.5 * ax * dt
        vy_new = vy_half + 0.5 * ay * dt

        return x_new, vx_new, y_new, vy_new



def adjust_acceleration(v, a, max_v, min_v, dt):
    # Predict the next velocity
    predicted_v = v + a * dt

    # Adjust acceleration if the predicted velocity exceeds limits
    if predicted_v > max_v:
        a = (max_v - v) / dt
    elif predicted_v < min_v:
        a = (min_v - v) / dt

    return a

# Define a function to update the plot based on the PID gains
def update_plot(Kp, Ki, Kd):

    vehicle = VehicleDynamics(mass=1, min_accel = -2, max_accel = 2)

    pid_x = PIDController(Kp, Ki, Kd, vehicle.min_accel, vehicle.max_accel)
    pid_y = PIDController(Kp, Ki, Kd, vehicle.min_accel, vehicle.max_accel)

    x, vx, y, vy = 0, 0, 0, 0
    all_x, all_y = [x], [y]
    acc_x, acc_y = [0], [0]
    v_x, v_y = [vx], [vy]
    times = [0]

    min_velocity, max_velocity = -1, 1
    dt = 0.02
    simulation_time = 20
    x_desired, y_desired = 1, 1
    final_desired = [0,0]
    import numpy as np
    for t in range(int(simulation_time / dt)):

        desired_position = np.array([x,y]) + np.array([vx*dt,vy*dt])
        error_x = x_desired - x
        error_y = y_desired - y

        ax = pid_x.update(error_x, dt)
        ay = pid_y.update(error_y, dt)

        # Adjust acceleration based on current velocity and limits
        ax = adjust_acceleration(vx, ax, max_velocity, min_velocity, dt)
        ay = adjust_acceleration(vy, ay, max_velocity, min_velocity, dt)




        # x, vx, y, vy = update_drone_state(x, vx, y, vy, ax, ay, dt)
        x, vx, y, vy = vehicle.update_state_verlet(x, vx, y, vy, ax, ay, dt)

        all_x.append(x)
        all_y.append(y)
        acc_x.append(ax)
        acc_y.append(ay)
        v_x.append(vx)
        v_y.append(vy)
        times.append(t * dt)

    # Create a figure with 3 subplots
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 15))

    # Position plot
    ax1.plot(times, all_x, label='X Position')
    ax1.plot(times, all_y, label='Y Position')
    ax1.set_title('Position over Time')
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Position (m)')
    ax1.legend()
    ax1.grid()

    # Velocity plot
    ax2.plot(times, v_x, label='X Velocity')
    ax2.plot(times, v_y, label='Y Velocity')
    ax2.set_title('Velocity over Time')
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Velocity (m/s)')
    ax2.legend()
    ax2.grid()

    # Acceleration plot
    ax3.plot(times, acc_x, label='X Acceleration')
    ax3.plot(times, acc_y, label='Y Acceleration')
    ax3.set_title('Acceleration over Time')
    ax3.set_xlabel('Time (s)')
    ax3.set_ylabel('Acceleration (m/s²)')
    ax3.legend()
    ax3.grid()

    # # Show the plot
    # plt.tight_layout()
    # plt.show()
    # plt.plot(times, v_x)

    # # Set the plot range for X and Y axes
    # plt.xlim(0,5)
    # plt.ylim(0,2)

    # plt.title("Drone Path")
    # plt.xlabel("Time (s)")
    # plt.ylabel("Y Position (m)")
    # plt.grid()
    plt.show()

# Simulation parameters

# min_accel, max_accel = -2.0, 2.0

# Create sliders for PID gains
Kp_slider = widgets.FloatSlider(value=2.4, min=0.1, max=40.0, step=0.1, description='Kp:')
Ki_slider = widgets.FloatSlider(value=0.5, min=0.01, max=1, step=0.01, description='Ki:')
Kd_slider = widgets.FloatSlider(value=4.5, min=0.1, max=40.0, step=0.1, description='Kd:')

# Create an interactive widget to update the plot
widgets.interactive(update_plot, Kp=Kp_slider, Ki=Ki_slider, Kd=Kd_slider)


interactive(children=(FloatSlider(value=2.4, description='Kp:', max=40.0, min=0.1), FloatSlider(value=0.5, des…

In [27]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

class PIDController:
    def __init__(self, Kp, Ki, Kd, min_output, max_output):
        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd
        self.min_output = min_output
        self.max_output = max_output
        self.cumulative_error = 0
        self.previous_error = 0

    def update(self, error, dt):
    
        # Proportional term
        P = self.Kp * error

        # Integral term
        self.cumulative_error += error * dt
        I = self.Ki * self.cumulative_error

        # Derivative term
        error_rate = (error - self.previous_error) / dt
        D = self.Kd * error_rate

        # Compute the control signal
        control = P + I + D

        # Clamp the output
        control = max(self.min_output, min(control, self.max_output))

        # Update previous error
        self.previous_error = error

        return control

class VehicleDynamics:
    def __init__(self, mass, min_accel, max_accel):
        self.mass = mass
        self.min_accel = min_accel
        self.max_accel = max_accel
        # Initialize other parameters specific to the vehicle dynamics

    # def update_state(self, x, vx, y, vy, force_x, force_y, dt):
    #     # Implement the dynamics here
    #     # For example, a simple point mass dynamics could be:
    #     ax = force_x / self.mass
    #     ay = force_y / self.mass

    #     # Update position and velocity (using Verlet or other integration method)
    #     # ...

    #     return new_x, new_vx, new_y, new_vy
    
    def update_state_verlet(self, x, vx, y, vy, ax, ay, dt):
        # Update position using Velocity Verlet
        x_new = x + vx * dt + 0.5 * ax * dt**2
        y_new = y + vy * dt + 0.5 * ay * dt**2

        # Predict the next velocity (half step)
        vx_half = vx + 0.5 * ax * dt
        vy_half = vy + 0.5 * ay * dt

        # Update acceleration here based on new position if needed
        # For constant acceleration, this step can be skipped

        # Complete the velocity update (second half step)
        vx_new = vx_half + 0.5 * ax * dt
        vy_new = vy_half + 0.5 * ay * dt

        return x_new, vx_new, y_new, vy_new



def adjust_acceleration(v, a, max_v, min_v, dt):
    # Predict the next velocity
    predicted_v = v + a * dt

    # Adjust acceleration if the predicted velocity exceeds limits
    if predicted_v > max_v:
        a = (max_v - v) / dt
    elif predicted_v < min_v:
        a = (min_v - v) / dt

    return a

# Define a function to update the plot based on the PID gains
def update_plot(Kp, Ki, Kd):

    vehicle = VehicleDynamics(mass=1, min_accel = -2, max_accel = 2)

    pid_x = PIDController(Kp, Ki, Kd, vehicle.min_accel, vehicle.max_accel)
    pid_y = PIDController(Kp, Ki, Kd, vehicle.min_accel, vehicle.max_accel)

    x, vx, y, vy = 0, 1, 0, 1
    all_x, all_y = [x], [y]
    acc_x, acc_y = [0], [0]
    v_x, v_y = [vx], [vy]
    times = [0]

    min_velocity, max_velocity = -1, 1
    dt = 0.02
    simulation_time = 7
    vx_desired, vy_desired = 0, 0

    
    for t in range(int(simulation_time / dt)):
        error_vx = vx_desired - vx
        error_vy = vy_desired - vy

        ax = pid_x.update(error_vx, dt)
        ay = pid_y.update(error_vy, dt)

        # Adjust acceleration based on current velocity and limits
        ax = adjust_acceleration(vx, ax, max_velocity, min_velocity, dt)
        ay = adjust_acceleration(vy, ay, max_velocity, min_velocity, dt)




        # x, vx, y, vy = update_drone_state(x, vx, y, vy, ax, ay, dt)
        x, vx, y, vy = vehicle.update_state_verlet(x, vx, y, vy, ax, ay, dt)

        all_x.append(x)
        all_y.append(y)
        acc_x.append(ax)
        acc_y.append(ay)
        v_x.append(vx)
        v_y.append(vy)
        times.append(t * dt)

    # Create a figure with 3 subplots
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 15))

    # Position plot
    ax1.plot(times, all_x, label='X Position')
    ax1.plot(times, all_y, label='Y Position')
    ax1.set_title('Position over Time')
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Position (m)')
    ax1.legend()
    ax1.grid()

    # Velocity plot
    ax2.plot(times, v_x, label='X Velocity')
    ax2.plot(times, v_y, label='Y Velocity')
    ax2.set_title('Velocity over Time')
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Velocity (m/s)')
    ax2.legend()
    ax2.grid()

    # Acceleration plot
    ax3.plot(times, acc_x, label='X Acceleration')
    ax3.plot(times, acc_y, label='Y Acceleration')
    ax3.set_title('Acceleration over Time')
    ax3.set_xlabel('Time (s)')
    ax3.set_ylabel('Acceleration (m/s²)')
    ax3.legend()
    ax3.grid()

    # # Show the plot
    # plt.tight_layout()
    # plt.show()
    # plt.plot(times, v_x)

    # # Set the plot range for X and Y axes
    # plt.xlim(0,5)
    # plt.ylim(0,2)

    # plt.title("Drone Path")
    # plt.xlabel("Time (s)")
    # plt.ylabel("Y Position (m)")
    # plt.grid()
    plt.show()

# Simulation parameters

# min_accel, max_accel = -2.0, 2.0

# Create sliders for PID gains
Kp_slider = widgets.FloatSlider(value=8, min=0.1, max=10.0, step=0.1, description='Kp:')
Ki_slider = widgets.FloatSlider(value=0.01, min=0.01, max=1, step=0.01, description='Ki:')
Kd_slider = widgets.FloatSlider(value=0.1, min=0.01, max=1.0, step=0.01, description='Kd:')

# Create an interactive widget to update the plot
widgets.interactive(update_plot, Kp=Kp_slider, Ki=Ki_slider, Kd=Kd_slider)


interactive(children=(FloatSlider(value=8.0, description='Kp:', max=10.0, min=0.1), FloatSlider(value=0.01, de…